NOTEBOOK TO LOAD ALL REQUIRED DATA 

In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/Response model final/response_model
/Users/shashankgupta/Documents/code/git_project/Response model final/response_model/code


QUERY FROM SNOWFLAKE FOR RAW DATA 

In [2]:
pilot = querySno("""select * from PROD_DB.ADHOC.PILOT_DATASET_3 """)
txn_features = querySno("""select * from PROD_DB.adhoc.lending_novo_txn_features_model_prescreen_v2 """)
risk_features = querySno("""select * from PROD_DB.ADHOC.lending_response_features_final """)

Common Features before removal 

In [3]:
os.chdir('./src/')
cwd = os.getcwd()
print(cwd)

from utils import Utils
print(f"utils module loaded ..")

ut = Utils()

/Users/shashankgupta/Documents/code/git_project/Response model final/response_model/code/src
utils module loaded ..


In [4]:
print(ut.intersection(pilot,txn_features))
print(ut.intersection(pilot,risk_features))
print(ut.intersection(risk_features,txn_features))

['business_id', 'email_sent_at', 'signed_on_x', 'past_1_month', 'fpd_plus_3']
['business_id', 'response_target']
['date_of_refresh', 'business_id']


In [5]:
txn_features = txn_features.drop(columns=['email_sent_at', 'signed_on_x', 'past_1_month', 'fpd_plus_3'],axis=1)
risk_features = risk_features.drop(columns=['response_target'],axis=1)
txn_features = txn_features.drop(columns=['date_of_refresh'],axis=1)

In [6]:
print(ut.intersection(pilot,txn_features))
print(ut.intersection(pilot,risk_features))
print(ut.intersection(risk_features,txn_features))

['business_id']
['business_id']
['business_id']


In [7]:
print(os.getcwd())

/Users/shashankgupta/Documents/code/git_project/Response model final/response_model/code/src


In [8]:
final_df = pd.merge(pd.merge(pilot,txn_features,on='business_id'),txn_features)
final_df = pd.merge(pd.merge(final_df,risk_features,on='business_id'),txn_features)

In [10]:
final_df.shape

(6261, 437)

In [9]:
final_df['response_target'].value_counts()

0    5477
1     784
Name: response_target, dtype: int64

In [12]:
from datetime import date
today = date.today()
final_df.to_pickle(f'/Users/shashankgupta/Documents/code/git_project/Response model final/response_model/data/novo_features_response_2022-12-11.pkl')

In [13]:
f'novo_features_response_{today}.pkl'

'novo_features_response_2022-12-11.pkl'

In [15]:
df = pd.read_pickle(f'/Users/shashankgupta/Documents/code/git_project/Response model final/response_model/data/novo_features_response_2022-12-11.pkl')

In [16]:
df.shape

(6261, 437)